In [1]:
from collections import Counter
import pandas as pd
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import numpy as np
from scipy.stats.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from scipy.stats.stats import pearsonr
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [2]:
already_extracted=True
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

if already_extracted:
    df = pd.read_pickle('../reviews_Kindle_Store_10_training.h5')
    Training_DataFrame = pd.read_pickle('../reviews_Kindle_Store_10_training.h5')
    Testing_DataFrame = pd.read_pickle('../reviews_Kindle_Store_10_testing.h5')
else:
    df = getDF('../reviews_Kindle_Store_5.json.gz')
    df = shuffle(df)
    Training_DataFrame, Testing_DataFrame = train_test_split(df, test_size=0.2)

In [3]:
print Training_DataFrame.shape
print Testing_DataFrame.shape

(453086, 13)
(50343, 13)


In [4]:
print "Dataset details:"
Uniq_items =  len(set(df['asin'].values))
print "Number of item:", Uniq_items
Uniq_users =   len(set(df['reviewerID'].values))
print "Number of Users:", Uniq_users
Uniq_ratings = len(set(df['overall'].values))
print "Number of ratings:", Uniq_ratings
print "\n"

Dataset details:
Number of item: 22203
Number of Users: 26466
Number of ratings: 5




In [5]:
#Training_DataFrame=Training_DataFrame[:5000]
Training_DataFrame.columns
# Lists of unique items and users
item_list = list(set(df['asin'].values))
user_list = list(set(df['reviewerID'].values))

#Storing the indices of users to generate the Rating Memory Matrix
UserIndices={}
for i in range(len(user_list)):
    UserIndices[user_list[i]] = i

#Storing the indices of items to generate the Rating Memory Matrix
#These indices will be used to populate the UsersRatingMemory for Memory based Collaborative Filtering
itemIndices={}
for j in range(len(item_list)):
    itemIndices[item_list[j]] = j
    

# Collaborative Filtering with Biases and without Sentiment Scores: 

In [6]:
#UsersRatingMemory stores the users and corresponding items ratings in Matrix form
UsersRatingMemory=np.zeros((Uniq_users,Uniq_items))

for i in range(0, len(Training_DataFrame)):
    item_id=Training_DataFrame.iloc[i]['asin']
    user_id=Training_DataFrame.iloc[i]['reviewerID']
    #get user and item indices
    user_ind=UserIndices[user_id]
    item_ind=itemIndices[item_id]
    rating=Training_DataFrame.iloc[i]['overall']
    #populate the matrix with rating at corresponding user and item index
    UsersRatingMemory[user_ind][item_ind]=float(rating)

AvgRatings = np.true_divide(UsersRatingMemory.sum(1),(UsersRatingMemory!=0).sum(1))

print "Average Ratings of the Users are:"
print AvgRatings

Average Ratings of the Users are:
[4.92307692 5.         3.76923077 ... 3.84615385 3.5        4.64912281]


In [23]:
# Your Code Here...
def populate_baseline():
    '''I have tried and implemented the Baseline appraoch of collaborative filtering as discussed in the class. 
       I observe that it models the Local and Global Effects of user rating very well.
       I stored the baseline ratings 

        b_xi= u + bx + bi
        u = overall mean rating
        bx = rating deviation of user x
            = (avg. rating of user x) – u
        bi = (avg. rating of movie i) – u'''
    
    AvgRatings = np.true_divide(UsersRatingMemory.sum(1),(UsersRatingMemory!=0).sum(1))
    AvgRatingitem= np.true_divide(UsersRatingMemory.sum(0),(UsersRatingMemory!=0).sum(0))
    #print AvgRatingitem
    global_average=np.true_divide(UsersRatingMemory.sum(),(UsersRatingMemory!=0).sum()) 

    baseline_matrix=np.zeros((len(AvgRatings),len(AvgRatingitem)))

    for i in range(len(AvgRatings)):
        for j in range(len(AvgRatingitem)): 
            baseline_matrix[i][j]=AvgRatings[i]+AvgRatingitem[j]-global_average
    return baseline_matrix

        
def PredictUserRatingBaseline(user_id, item_id):
    '''This function predicts the rating based on user_id and item_id'''
    item_ind = itemIndices[item_id]
    user_ind = UserIndices[user_id]
    #ratedindices: it stores the indices of all other users that have rated the item
    ratedindices=[]
    i=0
    for rating in UsersRatingMemory[:,item_ind]:
        if i==user_ind:
            continue
        if rating !=0:
            ratedindices.append(i)
        i+=1
    #itemRatingsOthers: stores other users ratings
    itemRatingsOthers = UsersRatingMemory[ratedindices, item_ind]
    #Store the pearson coefficents
    PearsonCoeffs=[]
    k=0.001
    for ind in ratedindices:
        PearsonCoeffs.append(pearsonr(UsersRatingMemory[ind],UsersRatingMemory[user_ind])[0])

    # Calculate the prediction
    pred = baseline_matrix[user_ind][item_ind] + k * np.sum( PearsonCoeffs* (itemRatingsOthers - baseline_matrix[[ratedindices],item_ind]))
    return pred

def TestBaseline(no_of_samples):
    '''This function returns the predictions for given no of samples'''
    Predictions=np.zeros(no_of_samples)
    for i in range(no_of_samples):
        item_id=Testing_DataFrame.iloc[i]['asin']
        user_id=Testing_DataFrame.iloc[i]['reviewerID']
        Predictions[i]=PredictUserRatingBaseline(user_id,item_id)
    return Predictions


In [ ]:
baseline_matrix=populate_baseline()
print baseline_matrix

In [24]:
Predictions=TestBaseline(10)
print "For 10 Samples:"
print "The Mean Absolute Error is:", mean_absolute_error(Predictions, Testing_DataFrame['overall'].values[:10])
print "Root Mean Square Error is:", np.sqrt(mean_squared_error(Predictions, Testing_DataFrame['overall'].values[:10]))

[[4.76347335 5.43807653 5.24918764 ... 4.54918764 3.86168764 4.96585431]
 [4.84039643 5.51499961 5.32611072 ... 4.62611072 3.93861072 5.04277738]
 [3.6096272  4.28423037 4.09534149 ... 3.39534149 2.70784149 3.81200815]
 ...
 [3.68655028 4.36115345 4.17226456 ... 3.47226456 2.78476456 3.88893123]
 [3.34039643 4.01499961 3.82611072 ... 3.12611072 2.43861072 3.54277738]
 [4.48951924 5.16412241 4.97523352 ... 4.27523352 3.58773352 4.69190019]]
For 10 Samples:
The Mean Absolute Error is: 0.5426470836682358
Root Mean Square Error is: 0.6192298296902573


In [18]:
def get_other_closest_user(user_ind,k):
    PearsonCoeffs={}
    for ind in range(len(user_list)):
        if ind==user_ind :
            continue
        else:
            PearsonCoeffs[ind]=pearsonr(UsersRatingMemory[ind],UsersRatingMemory[user_ind])[0]
        
    PearsonCoeffs_sorted=sorted(PearsonCoeffs.items(), key=lambda kv: kv[1], reverse=True)
    return [x for x, y in PearsonCoeffs_sorted[:k]]

In [19]:
def recommender(user_ind):
    items_to_recommend=[]
    colsest_users=get_other_closest_user(user_ind,20)
    for user in colsest_users:
        for items in np.nonzero(UsersRatingMemory[user]):
            for item in items:
                if UsersRatingMemory[user_ind][item] == 0:
                    items_to_recommend.append(item_list[item])
    
    recommendations = {}
    for item in items_to_recommend:
        recommendations[item] = PredictUserRatingBaseline(user_list[user_ind], item)
        
    sorted_recommendations = sorted(recommendations.items(), key=lambda kv: kv[1], reverse=True)
    return sorted_recommendations

In [20]:
print "getting recommendations for user:", user_list[0]
print "Top 100 recommendations are For Collaborative Filtering without sentiment Scores ..."
recommendations=recommender(0)
print recommendations[:100]

getting recommendations for user: AO00I2TVC9LBB
Top 100 recommendations are For Collaborative Filtering without sentiment Scores ...
[('B00KJ7RAYI', 5.5491876393647805), ('B00G66WM4Y', 5.5491876393647805), ('B00LPUXAGK', 5.507520972698113), ('B00A1Q7SQM', 5.496556060417412), ('B00KEXI780', 5.493632083809225), ('B00KWREVQK', 5.482520972698114), ('B00GMSI5BA', 5.472264562441704), ('B00HBY2SSU', 5.472264562441704), ('B00GXLDT9O', 5.465854306031448), ('B00F3OO3IS', 5.465854306031448), ('B00IX330GW', 5.465854306031448), ('B00HSNMKF0', 5.465854306031448), ('B00LYPZJ4M', 5.46223111762565), ('B00KVN6HH6', 5.46223111762565), ('B00KKPKAAU', 5.442804660641376), ('B00JHNAWQS', 5.442044782221923), ('B00JYQARIG', 5.43807652825367), ('B00DDHE79C', 5.4315405805412516), ('B00K3517K4', 5.4201553813002645), ('B00IO05YO0', 5.415854306031448), ('B00JK54RAU', 5.406330496507638), ('B00JF8GMP0', 5.394015225571677), ('B00KF076S4', 5.38918763936478), ('B00KKT5TMU', 5.37775906793621), ('B00JCJAK96', 5.3706162107

# Colloboarative Filtering with biases and Sentiment Scores:

In [25]:
UsersRatingMemory_sentiment=np.zeros((Uniq_users,Uniq_items))

for i in range(0, len(Training_DataFrame)):
    item_id=Training_DataFrame.iloc[i]['asin']
    user_id=Training_DataFrame.iloc[i]['reviewerID']
    #get user and item indices
    user_ind=UserIndices[user_id]
    item_ind=itemIndices[item_id]
    rating=float(Training_DataFrame.iloc[i]['overall'])
    
    if Training_DataFrame.iloc[i]['reviewText']:
    #populate the matrix with rating at corresponding user and item index
        rating=rating + float(Training_DataFrame.iloc[i]['corrected_sent_score'])
    
    UsersRatingMemory_sentiment[user_ind][item_ind]=rating

AvgRating_sentiment = np.true_divide(UsersRatingMemory_sentiment.sum(1),(UsersRatingMemory_sentiment!=0).sum(1))

print "Average Ratings of the Users are:"
print AvgRating_sentiment

Average Ratings of the Users are:
[5.19210769 5.352625   4.11755385 ... 4.19432308 3.7305625  4.78263509]


In [22]:

def PredictUserRatingsentiment(user_id, item_id):
    '''This function predicts the rating based on user_id and item_id'''
    item_ind = itemIndices[item_id]
    user_ind = UserIndices[user_id]
    #ratedindices: it stores the indices of all other users that have rated the item
    ratedindices=[]
    i=0
    for rating in UsersRatingMemory_sentiment[:,item_ind]:
        if i==user_ind:
            continue
        if rating !=0:
            ratedindices.append(i)
        i+=1
    #itemRatingsOthers: stores other users ratings
    itemRatingsOthers = UsersRatingMemory_sentiment[ratedindices, item_ind]
    #Store the pearson coefficents
    PearsonCoeffs=[]
    for ind in ratedindices:
        PearsonCoeffs.append(pearsonr(UsersRatingMemory_sentiment[ind],UsersRatingMemory_sentiment[user_ind])[0])
    k=0.001
    # Calculate the prediction
    pred = AvgRating_sentiment[user_ind] + k * np.sum( PearsonCoeffs* (itemRatingsOthers - AvgRating_sentiment[ratedindices]))
    return pred

def Testsentiment(no_of_samples):
    '''This function returns the predictions for given no of samples'''
    Predictions=np.zeros(no_of_samples)
    for i in range(no_of_samples):
        item_id=Testing_DataFrame.iloc[i]['asin']
        user_id=Testing_DataFrame.iloc[i]['reviewerID']
        Predictions[i]=PredictUserRatingsentiment(user_id,item_id)
    return Predictions

In [23]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
Predictions=Testsentiment(1000)
print "For 1000 Samples:"
print "The Mean Absolute Error is:", mean_absolute_error(Predictions, Testing_DataFrame['overall'].values[:1000])
print "Root Mean Square Error is:", np.sqrt(mean_squared_error(Predictions, Testing_DataFrame['overall'].values[:1000]))

KeyboardInterrupt: 

In [24]:
def get_other_closest_user_sentiment(user_ind,k):
    PearsonCoeffs={}
    for ind in range(len(user_list)):
        if ind==user_ind :
            continue
        else:
            PearsonCoeffs[ind]=pearsonr(UsersRatingMemory_sentiment[ind],UsersRatingMemory_sentiment[user_ind])[0]
        
    PearsonCoeffs_sorted=sorted(PearsonCoeffs.items(), key=lambda kv: kv[1], reverse=True)
    return [x for x, y in PearsonCoeffs_sorted[:k]]

In [25]:
def recommender_sentiments(user_ind):
    items_to_recommend=[]
    colsest_users=get_other_closest_user_sentiment(user_ind,20)
    for user in colsest_users:
        for items in np.nonzero(UsersRatingMemory_sentiment[user]):
            for item in items:
                if UsersRatingMemory_sentiment[user_ind][item] == 0:
                    items_to_recommend.append(item_list[item])
    
    recommendations = {}
    for item in items_to_recommend:
        recommendations[item] = PredictUserRatingsentiment(user_list[user_ind], item)
        
    sorted_recommendations = sorted(recommendations.items(), key=lambda kv: kv[1], reverse=True)
    return sorted_recommendations

In [26]:
print "getting recommendations for user:", user_list[1]
print "Top 50 recommendations are For Collaborative Filtering with sentiment Scores ..."
recommendations_sentiment=recommender_sentiments(1)


getting recommendations for user: AO00I2TVC9LBB
Top 50 recommendations are For Collaborative Filtering with sentiment Scores ...
[('B00KIXIBYQ', 5.240565517241379), ('B00KBDWHKM', 5.240565517241379), ('B00BTPOFSI', 5.240565517241379), ('B00FR3ZCG2', 5.240565517241379), ('B00GW7NHXM', 5.240565517241379), ('B00IW3BOC0', 5.240565517241379), ('B00BVR73P6', 5.240565517241379), ('B00AMIH8Y6', 5.240565517241379), ('B00CMXCWW2', 5.240565517241379), ('B00GM4Y0OU', 5.240565517241379), ('B00KLPFCJI', 5.240565517241379), ('B00FK6WG6K', 5.240565517241379), ('B00HHEVSK8', 5.240565517241379), ('B0058PIWXY', 5.240565517241379), ('B0089MXR6K', 5.240565517241379), ('B004PLNH5K', 5.240565517241379), ('B0069F1LLM', 5.240565517241379), ('B00K0YBVIG', 5.240565517241379), ('B00KXIIZKG', 5.240565517241379), ('B00IZCODEE', 5.240565517241379), ('B008EKOPGS', 5.240565517241379), ('B00F7CJKAC', 5.240565517241379), ('B008OPADK0', 5.240565517241379), ('B00CYBFTWC', 5.240565517241379), ('B00BVA2QBO', 5.2405655172413

1
